In [1]:
import open3d as o3d
import cl
import utils as ut
import numpy as np
from skimage.morphology import binary_dilation
import proc3d
import json
from PIL import Image
from utils import *
import glob
import os
%matplotlib widget 
#%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import glob

In [2]:
class space_carving():
    def __init__(self, dataset_path):
        self.images = self.load_images(os.path.join(dataset_path, 'imgs'))
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        #self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.bbox = json.load(open(os.path.join(dataset_path, '/home/pico/uni/romi/scanner_cube/bbox_min_max.json')))
        
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))
        self.gt=o3d.io.read_point_cloud(params["gt_path"])
        self.gt_points = np.asarray(self.gt.points)
        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                                                                                                                                                                                                  
        return ext 
    
    def load_images(self,path):                                                                                                                                         
        imgs = []
        img_files = glob.glob(os.path.join(path, '*.png')) #get all .png files from folder path
        assert len(img_files) != 0,"Image list is empty."
        for i in sorted(img_files):                                                                                                                                     
            img = Image.open(i)                                                                                                                                      
            imgs.append(img.copy())                                                                                                                                     
            img.close()                                                                                                                                                                                                                                                   
        return imgs

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1
        
        print(nx,ny,nz)

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        self.space_carve(self.images[idx], self.extrinsics[idx])
        
    def space_carve(self, im, rt):
        mask = get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)
        
    def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd

In [3]:
class space_carving_2():
    def __init__(self, dataset_path):
        self.img_files = sorted (glob.glob(os.path.join(dataset_path, 'imgs', '*.png')) )#get all .png file names from folder path
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        #self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.bbox = json.load(open(os.path.join(dataset_path, '/home/pico/uni/romi/scanner_cube/bbox_min_max.json')))
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))
        self.gt=o3d.io.read_point_cloud(params["gt_path"])
        self.gt_points = np.asarray(self.gt.points)
        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                                                                                                                                                                                                  
        return ext 
    
    def load_image(self,idx):                                                                                                                                         
        img = Image.open(self.img_files[idx])                                                                                                                                      
        cp = img.copy()                                                                                                                                 
        img.close()                                                                                                                                                                                                                                                   
        return cp

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        im = self.load_image(idx)
        self.space_carve(im, self.extrinsics[idx])
        
    def space_carve(self, im, rt):
        mask = get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)
        
    def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd

In [5]:
class space_carving_2_masks():
    def __init__(self, dataset_path,gt_mode=False):
        self.masks_files = sorted (glob.glob(os.path.join(dataset_path, 'masks', '*.png')) )#get all .png file names from folder path
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        #self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.bbox = json.load(open(os.path.join(dataset_path, '/home/pico/uni/romi/scanner_cube/bbox_min_max.json')))
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))


        #self.gt=o3d.io.read_point_cloud(params["gt_path"])
        #self.gt_points = np.asarray(self.gt.points)
        self.gt_mode = gt_mode

        if self.gt_mode is True:
            self.gt = np.load(os.path.join(dataset_path, 'volumes','vol_180.npy'))
            self.gt_solid_mask = np.where(self.gt==1,True,False) 
            self.gt_n_solid_voxels = np.count_nonzero(self.gt_solid_mask)


        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                         
        return ext 
    
    def load_mask(self,idx):                               
        img = cv2.imread(self.masks_files[idx], cv2.IMREAD_GRAYSCALE)
        return img
    

        

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        im = self.load_mask(idx)
        self.space_carve(im, self.extrinsics[idx])
        
    def space_carve(self, mask, rt):
        #mask = im.copy() #get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)


    def gt_compare(self,test_vol):
        if self.gt_mode is False:
            return 0
        #compare current volume with ground truth (voxelwise) and return percentage
        comp = np.where( self.gt==test_vol,True,False)
        eq_count = np.count_nonzero(comp)
        #perc_sim = (eq_count/np.prod(gt_vol.shape) )*100.
        #perc_sim = (eq_count/682176)*100. #682176number of voxels of the volumes used here 
        perc_sim = eq_count * 0.00014658973637301812
        return perc_sim
    
    def gt_compare_solid(self,test_vol):
        if self.gt_mode is False:
            return 0
        #compares only solid voxels (with 1;s) between ground truth and test_vol  
        vol_solid_mask = np.where(test_vol==1,True,False) 
        vol_n_solid_voxels = np.count_nonzero(vol_solid_mask)
        intersection = self.gt_solid_mask & vol_solid_mask
        n_intersection = np.count_nonzero(intersection)
        ratio = n_intersection / ( self.gt_n_solid_voxels + vol_n_solid_voxels - n_intersection )
        return ratio
    
    '''def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd'''


In [6]:
#data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/000'
#spc = space_carving(data_path)

#for i in range(180):
#    spc.carve(i)

# for creating a responsive plot
%matplotlib widget
  
# importing required libraries
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
  
x,y,z = np.where(spc.sc.values()[:64,:64,:128]==1)
  
# creating figure
fig = plt.figure()
ax = Axes3D(fig)

ax.set_xlim3d(0, 64)
ax.set_ylim3d(0, 64)
ax.set_zlim3d(0, 128)

#ax.set_xlim3d(0, 140)
#ax.set_ylim3d(0, 140)
#ax.set_zlim3d(0, 140)
  
# creating the plot
plot_geeks = ax.scatter(x, y, z, color='green',s=1)

x,y,z = np.where(spc.sc.values()==0)
#plot_geeks = ax.scatter(x, y, z, color='red',s=1)
  
# setting title and labels
ax.set_title("3D plot")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
ax.grid()
  
# displaying the plot
plt.show()

NameError: name 'spc' is not defined

In [7]:
np.histogram(spc.sc.values(), bins=3)

NameError: name 'spc' is not defined

In [8]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204'#'
spc = space_carving_2_masks(data_path)

In [9]:
spc.carve(45)
print(np.histogram(spc.sc.values(), bins=3)[0])

[113607 333262 235307]


In [ ]:
print(np.histogram(spc.sc.values(), bins=3)[0])

In [ ]:
ant = 0
for i in range(0,180,40):
    spc.carve(i)
    h = np.histogram(spc.sc.values(), bins=3)
    print(i,h[0],h[0][0]-ant)
    ant=h[0][0]

In [17]:

pos = np.random.randint(180, size=10)
for i in range(180):#pos:
    spc.carve(i)
    print(i,np.histogram(spc.sc.values(), bins=3)[0])


0 [395109 177999 109068]
1 [401372 177347 103457]
2 [405121 176853 100202]
3 [408838 176336  97002]
4 [412852 175710  93614]
5 [417207 175063  89906]
6 [421805 174403  85968]
7 [426864 173684  81628]
8 [431724 172998  77454]
9 [435703 172204  74269]
10 [440022 171372  70782]
11 [442541 170608  69027]
12 [444887 169782  67507]
13 [447230 168870  66076]
14 [449857 168009  64310]
15 [451509 167090  63577]
16 [452835 166167  63174]
17 [454636 165190  62350]
18 [456058 164229  61889]
19 [457597 163273  61306]
20 [459011 162313  60852]
21 [460459 161278  60439]
22 [461766 160367  60043]
23 [463308 159275  59593]
24 [464943 158168  59065]
25 [466206 157308  58662]
26 [467635 156291  58250]
27 [468937 155276  57963]
28 [470059 154312  57805]
29 [471731 153309  57136]
30 [473058 152341  56777]
31 [474272 151349  56555]
32 [475812 150348  56016]
33 [477027 149384  55765]
34 [478291 148399  55486]
35 [479667 147400  55109]
36 [481071 146419  54686]
37 [482401 145446  54329]
38 [483709 144447  540

In [ ]:
spc.sc.values().shape

In [ ]:
#data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/'
data_path = '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets'
gdeltas = []
for plantn in [202]:#range(180):
    spc = space_carving_2(os.path.join(data_path,str(plantn).zfill(3)))
    for i in range(180):
        deltas = []
        pos = np.random.randint(180, size=10)
        n_ant = 0
        for p in pos:
            spc.carve(p)
            espaces = np.histogram(spc.sc.values(), bins=3)[0][0]
            delta = espaces-n_ant
            deltas.append(delta)
            n_ant = espaces
        print(deltas)
        gdeltas.append(deltas)

In [21]:
data =[]
for bias in range(180):
    data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204'
    spc = space_carving_2_masks(data_path)
    for i in range(0,180,10):
        spc.carve(calculate_position(i,bias))
    h = np.histogram(spc.sc.values(), bins=3)[0]
    data.append(h)
    print(bias,h)

print(np.mean(data,axis=0))

0 [626601  11215  44360]
1 [626418  11216  44542]
2 [628229  11210  42737]
3 [629678  11208  41290]
4 [628930  11219  42027]
5 [628003  11229  42944]
6 [629223  11225  41728]
7 [629897  11223  41056]
8 [629682  11218  41276]
9 [629757  11220  41199]
10 [626601  11215  44360]
11 [626418  11216  44542]
12 [628229  11210  42737]
13 [629678  11208  41290]
14 [628930  11219  42027]
15 [628003  11229  42944]
16 [629223  11225  41728]
17 [629897  11223  41056]
18 [629682  11218  41276]
19 [629757  11220  41199]
20 [626601  11215  44360]
21 [626418  11216  44542]
22 [628229  11210  42737]
23 [629678  11208  41290]
24 [628930  11219  42027]
25 [628003  11229  42944]
26 [629223  11225  41728]
27 [629897  11223  41056]
28 [629682  11218  41276]
29 [629757  11220  41199]
30 [626601  11215  44360]
31 [626418  11216  44542]
32 [628229  11210  42737]
33 [629678  11208  41290]
34 [628930  11219  42027]
35 [628003  11229  42944]
36 [629223  11225  41728]
37 [629897  11223  41056]
38 [629682  11218  412

In [ ]:
dt = np.array(gdeltas)

In [ ]:
dt.shape

In [22]:
data =[]
for n in range(180):
    data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204'
    spc = space_carving_2_masks(data_path)
    pos = np.random.randint(180, size=18)
    for i in pos:
        spc.carve(i)
    h = np.histogram(spc.sc.values(), bins=3)[0]
    data.append(h)
    print(h)

print(np.mean(data,axis=0))

[617096  12367  52713]
[617250  11747  53179]
[627924  11369  42883]
[621568  11561  49047]
[632480  11414  38282]
[631326  11335  39515]
[628576  11470  42130]
[626794  11656  43726]
[629157  11692  41327]
[617627  12265  52284]
[603710  11870  66596]
[617024  12008  53144]
[624652  12398  45126]
[627197  11545  43434]
[622260  11399  48517]
[618488  12459  51229]
[623949  11468  46759]
[627823  11436  42917]
[622019  11487  48670]
[619089  11359  51728]
[630199  11497  40480]
[629490  11621  41065]
[619368  11579  51229]
[620102  12550  49524]
[626514  11481  44181]
[607229  13789  61158]
[625852  12392  43932]
[620274  11971  49931]
[622398  11448  48330]
[626854  12242  43080]
[622952  11556  47668]
[628466  11491  42219]
[624183  11524  46469]
[633119  11638  37419]
[606946  12029  63201]
[608881  11457  61838]
[619234  12226  50716]
[631637  11492  39047]
[622741  11633  47802]
[610021  11654  60501]
[620471  11643  50062]
[626376  11538  44262]
[600725  11817  69634]
[628203  11

In [ ]:
np.mean(dt,axis=0)

In [ ]:
np.min(dt,axis=0)

In [ ]:
np.max(dt,axis=0)

In [ ]:
np.std(dt,axis=0)

In [3]:
def calculate_position(init_state,steps):
        n_positions = 180
        n_pos = init_state + steps
        if n_pos>(n_positions-1):
            n_pos -= n_positions
        elif n_pos<0:
            n_pos += n_positions
        return n_pos

In [ ]:
rel_pos = 0
p_bias = 179
abs_pos = calculate_position(rel_pos,p_bias)
print(abs_pos)

In [ ]:
spc.sc.values().shape

In [5]:
spc.sc.values().shape

(66, 68, 152)

In [6]:
66*68*152

682176

In [23]:
def test_uniform(data_path,n_images):
    data =[]
    for bias in range(180):
        spc = space_carving_2_masks(data_path)
        dist = 180//n_images
        for i in range(0,180,dist):
            spc.carve(calculate_position(i,bias))
        h = np.histogram(spc.sc.values(), bins=3)[0]
        data.append(h)
        print(bias,h)
    return data

def test_random(data_path,n_images):
    data =[]
    for n in range(180):
        spc = space_carving_2_masks(data_path)
        pos = np.random.randint(180, size=n_images)
        for i in pos:
            spc.carve(i)
        h = np.histogram(spc.sc.values(), bins=3)[0]
        data.append(h)
        print(h)
    return data

def test_all(data_path):
    spc = space_carving_2_masks(data_path)
    for i in range(180):#pos:
        spc.carve(i)
        count = np.histogram(spc.sc.values(), bins=3)[0]
        print(i,count)
    return count

In [4]:
def compare_vols(gt_vol,test_vol):
    comp = np.where( gt_vol==test_vol,True,False)
    eq_count = np.count_nonzero(comp)
    #perc_sim = (eq_count/np.prod(gt_vol.shape) )*100.
    #perc_sim = (eq_count/682176)*100. #682176number of voxels of the volumes used here 
    perc_sim = eq_count * 0.00014658973637301812
    return perc_sim 

def test_uniform(data_path,n_images):
    data =[]
    gt_dist = []
    for bias in range(180):
        spc = space_carving_2_masks(data_path,gt_mode=True)
        dist = 180//n_images
        for i in range(0,180,dist):
            spc.carve(calculate_position(i,bias))
        gt_sim = spc.gt_compare_solid(spc.sc.values())
        gt_dist.append(gt_sim)
        h = np.histogram(spc.sc.values(), bins=3)[0]
        data.append(h)
        print(bias,h,gt_sim)
    return data,gt_dist

def test_random(data_path,n_images):
    data =[]
    gt_dist = []
    for n in range(180):
        spc = space_carving_2_masks(data_path,gt_mode=True)
        pos = np.random.randint(180, size=n_images)
        for i in pos:
            spc.carve(i)
        gt_sim = spc.gt_compare_solid(spc.sc.values())
        gt_dist.append(gt_sim)
        h = np.histogram(spc.sc.values(), bins=3)[0]
        data.append(h)
        print(h,gt_sim)
    return data,gt_dist

def test_all(data_path):
    spc = space_carving_2_masks(data_path)
    for i in range(180):#pos:
        spc.carve(i)
        count = np.histogram(spc.sc.values(), bins=3)[0]
        print(i,count)
    return count

In [9]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/000'
#print('count_rand: mean ',np.mean(count_rand,axis=0), ' std ',np.std(count_rand,axis=0) )
#print('count_uni', np.mean(count_uni,axis=0), ' std ',np.std(count_uni,axis=0) )
#print('count_all',count_all)

count_rand,dist_rand = test_random(data_path,20)
count_uni, dist_uni = test_uniform(data_path,20)
count_all = test_all(data_path)

print('count_rand',np.mean(count_rand,axis=0),np.mean(dist_rand))
print('count_uni', np.mean(count_uni,axis=0),np.mean(dist_uni))
print('count_all',count_all)

[667514  11412   3250] 0.730319803198032
[666764  11740   3672] 0.6464344039194339
[667428  11361   3387] 0.7007966951903216
[667310  11554   3312] 0.7166566083283041
[666185  12437   3554] 0.6683544303797468
[667320  11506   3350] 0.7085322195704057
[667109  11512   3555] 0.6676974978914816
[667459  11368   3349] 0.7092537313432836
[667212  11454   3510] 0.67625284738041
[666558  11578   4040] 0.5875804057397328
[666609  11808   3759] 0.6319148936170212
[666785  11571   3820] 0.6214024071166928
[665209  12393   4574] 0.519344262295082
[667019  11606   3551] 0.6684491978609626
[667215  11450   3511] 0.6760603472815258
[667026  11516   3634] 0.6531903190319032
[667209  11432   3535] 0.6714729997172745
[667673  11320   3183] 0.7456828885400314
[667542  11345   3289] 0.7221884498480243
[667511  11362   3303] 0.7191283292978208
[667131  11571   3474] 0.6832566168009206
[667371  11424   3381] 0.7020396098137748
[667501  11479   3196] 0.7426516572858036
[667075  11454   3647] 0.6508632502055

15 [667960  11194   3022] 0.7859741978167384
16 [667972  11191   3013] 0.7883211678832117
17 [667946  11185   3045] 0.7800393959290873
18 [667963  11199   3014] 0.7874668435013262
19 [667957  11201   3018] 0.7864238410596026
20 [667958  11204   3014] 0.7874668435013262
21 [667949  11204   3023] 0.7851239669421488
22 [667974  11198   3004] 0.790086493679308
23 [667977  11201   2998] 0.7916666666666666
24 [667960  11194   3022] 0.7859741978167384
25 [667972  11191   3013] 0.7883211678832117
26 [667946  11185   3045] 0.7800393959290873
27 [667963  11199   3014] 0.7874668435013262
28 [667957  11201   3018] 0.7864238410596026
29 [667958  11204   3014] 0.7874668435013262
30 [667949  11204   3023] 0.7851239669421488
31 [667974  11198   3004] 0.790086493679308
32 [667977  11201   2998] 0.7916666666666666
33 [667960  11194   3022] 0.7859741978167384
34 [667972  11191   3013] 0.7883211678832117
35 [667946  11185   3045] 0.7800393959290873
36 [667963  11199   3014] 0.7874668435013262
37 [667957  

31 [524169 151349   6658]
32 [525225 150348   6603]
33 [526227 149384   6565]
34 [527270 148399   6507]
35 [528329 147400   6447]
36 [529361 146419   6396]
37 [530400 145446   6330]
38 [531484 144447   6245]
39 [532511 143442   6223]
40 [533476 142457   6243]
41 [534470 141459   6247]
42 [535491 140450   6235]
43 [536592 139432   6152]
44 [537723 138356   6097]
45 [538753 137340   6083]
46 [539896 136250   6030]
47 [541048 135133   5995]
48 [542163 134049   5964]
49 [543488 132872   5816]
50 [544634 131745   5797]
51 [545837 130511   5828]
52 [546998 129317   5861]
53 [548208 128075   5893]
54 [549478 126790   5908]
55 [550815 125514   5847]
56 [552218 124144   5814]
57 [553609 122782   5785]
58 [555121 121286   5769]
59 [556656 119753   5767]
60 [558211 118230   5735]
61 [559866 116569   5741]
62 [561581 114835   5760]
63 [563344 113032   5800]
64 [565246 111039   5891]
65 [567171 109078   5927]
66 [569293 106855   6028]
67 [571760 104280   6136]
68 [574090 101809   6277]
69 [576615  

In [5]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204'
#print('count_rand: mean ',np.mean(count_rand,axis=0), ' std ',np.std(count_rand,axis=0) )
#print('count_uni', np.mean(count_uni,axis=0), ' std ',np.std(count_uni,axis=0) )
#print('count_all',count_all)

count_rand,dist_rand = test_random(data_path,5)
count_uni, dist_uni = test_uniform(data_path,5)
count_all = test_all(data_path)

print('count_rand',np.mean(count_rand,axis=0),np.mean(dist_rand))
print('count_uni', np.mean(count_uni,axis=0),np.mean(dist_uni))
print('count_all',count_all)

[407549  16988 257639] 0.19794225746232452
[249493 196741 235942] 0.21614314047905614
[446401  13710 222065] 0.22964874125905613
[437502  19209 225465] 0.226185988549229
[389281  41944 250951] 0.2032170791280337
[503542  17505 161129] 0.3165021191568052
[490456  14024 177696] 0.28698274794899786
[336573  35048 310555] 0.1642169252713498
[463616  14505 204055] 0.24993384752589748
[524255  14693 143228] 0.3561674439386746
[433602  93210 155364] 0.3236193859947283
[453616  15268 213292] 0.23909354285232098
[459154  17015 206007] 0.24754767532725974
[478819  13629 189728] 0.2687852437417655
[412298  22006 247872] 0.205741163561845
[479984  16752 185440] 0.2749997304029936
[517734  38034 126408] 0.4034121918230786
[439385  15966 226825] 0.2248299514650844
[375155  20978 286043] 0.17828815129428627
[470071  27514 184591] 0.27627131497410784
[516918  15153 150105] 0.33973462291674994
[463884  37654 180638] 0.2823093103066534
[509693  21185 151298] 0.3370649889636395
[530378  14021 137777] 0.3

14 [505025  12759 164392] 0.31022863433711856
15 [512984  12713 156479] 0.32591470817518436
16 [507790  12725 161661] 0.3154606911139837
17 [510148  12711 159317] 0.3201014202602032
18 [519685  12737 149754] 0.34052184595662854
19 [516252  12732 153192] 0.3328895067650917
20 [512627  12703 156846] 0.3251268582502486
21 [506322  12705 163149] 0.31256779697372694
22 [497146  12697 172333] 0.29591250616460213
23 [487369  12698 182109] 0.28004326603671026
24 [479312  12695 190169] 0.26820201382864056
25 [468400  12685 201091] 0.2536361240533646
26 [463000  12673 206503] 0.24700749578725134
27 [455110  12701 214365] 0.23796040601559876
28 [451297  12679 218200] 0.23378962370937953
29 [449044  12685 220447] 0.23140672061545162
30 [454242  12684 215250] 0.23698782867230325
31 [449752  12713 219711] 0.23216504416924946
32 [444906  12705 224565] 0.22712517977923333
33 [445372  12704 224100] 0.22756355913297222
34 [448473  12704 220999] 0.23075635346867493
35 [458455  12690 211031] 0.24165493975

26 [454343 156291  71542]
27 [455648 155276  71252]
28 [457026 154312  70838]
29 [458315 153309  70552]
30 [459779 152341  70056]
31 [461098 151349  69729]
32 [462475 150348  69353]
33 [463762 149384  69030]
34 [464990 148399  68787]
35 [466205 147400  68571]
36 [467344 146419  68413]
37 [468703 145446  68027]
38 [469766 144447  67963]
39 [470850 143442  67884]
40 [471952 142457  67767]
41 [473121 141459  67596]
42 [474103 140450  67623]
43 [475434 139432  67310]
44 [476627 138356  67193]
45 [477827 137340  67009]
46 [479084 136250  66842]
47 [480364 135133  66679]
48 [481478 134049  66649]
49 [482679 132872  66625]
50 [483871 131745  66560]
51 [485160 130511  66505]
52 [486500 129317  66359]
53 [487834 128075  66267]
54 [489318 126790  66068]
55 [490677 125514  65985]
56 [492248 124144  65784]
57 [493772 122782  65622]
58 [495382 121286  65508]
59 [496986 119753  65437]
60 [498389 118230  65557]
61 [499528 116569  66079]
62 [502378 114835  64963]
63 [503704 113032  65440]
64 [505030 1

In [10]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/203'
#print('count_rand: mean ',np.mean(count_rand,axis=0), ' std ',np.std(count_rand,axis=0) )
#print('count_uni', np.mean(count_uni,axis=0), ' std ',np.std(count_uni,axis=0) )
#print('count_all',count_all)

count_rand,dist_rand = test_random(data_path,10)
count_uni, dist_uni = test_uniform(data_path,10)
count_all = test_all(data_path)

print('count_rand',np.mean(count_rand,axis=0),np.mean(dist_rand))
print('count_uni', np.mean(count_uni,axis=0),np.mean(dist_uni))
print('count_all',count_all)

[662611  13732   5833] 0.5678038745071147
[663653  13017   5506] 0.6015256084271704
[655580  20596   6000] 0.552
[663762  12586   5828] 0.5682910089224433
[664739  12292   5145] 0.643731778425656
[664918  12267   4991] 0.663594470046083
[663183  13219   5774] 0.5736058191894701
[661395  14876   5905] 0.5608806096528366
[664559  12264   5353] 0.618718475621147
[665046  11857   5273] 0.628105442821923
[662623  13717   5836] 0.5675119945167924
[663457  12535   6184] 0.535575679172057
[662834  13955   5387] 0.6148134397623909
[665625  11736   4815] 0.6878504672897197
[661519  14473   6184] 0.535575679172057
[664567  12219   5390] 0.6144712430426716
[663709  12616   5851] 0.5660570842591011
[662993  13528   5655] 0.5856763925729443
[664413  12454   5309] 0.6238462987379921
[664493  12435   5248] 0.6310975609756098
[665006  12088   5082] 0.6517119244391971
[665170  12150   4856] 0.6820428336079077
[663935  12631   5610] 0.5903743315508021
[663773  12827   5576] 0.593974175035868
[663218  135

16 [666045  11466   4665] 0.7099678456591639
17 [666086  11448   4642] 0.7134855665661353
18 [666097  11457   4622] 0.7165729121592385
19 [666068  11464   4644] 0.7131782945736435
20 [666054  11457   4665] 0.7099678456591639
21 [666062  11460   4654] 0.7116458960034379
22 [666059  11456   4661] 0.7105771293713795
23 [666163  11457   4556] 0.7269534679543459
24 [666124  11452   4600] 0.72
25 [666128  11447   4601] 0.7198435122799391
26 [666083  11437   4656] 0.711340206185567
27 [666053  11460   4663] 0.7102723568518121
28 [666034  11455   4687] 0.7066353744399403
29 [665974  11466   4736] 0.6993243243243243
30 [666005  11470   4701] 0.7045309508615188
31 [666016  11480   4680] 0.7076923076923077
32 [665990  11477   4709] 0.7033340411977065
33 [666017  11472   4687] 0.7066353744399403
34 [666045  11466   4665] 0.7099678456591639
35 [666086  11448   4642] 0.7134855665661353
36 [666097  11457   4622] 0.7165729121592385
37 [666068  11464   4644] 0.7131782945736435
38 [666054  11457   4665]

38 [530077 144447   7652]
39 [531260 143442   7474]
40 [532415 142457   7304]
41 [533588 141459   7129]
42 [534741 140450   6985]
43 [535891 139432   6853]
44 [537088 138356   6732]
45 [538209 137340   6627]
46 [539400 136250   6526]
47 [540623 135133   6420]
48 [541814 134049   6313]
49 [543098 132872   6206]
50 [544303 131745   6128]
51 [545633 130511   6032]
52 [546907 129317   5952]
53 [548213 128075   5888]
54 [549577 126790   5809]
55 [550925 125514   5737]
56 [552366 124144   5666]
57 [553790 122782   5604]
58 [555361 121286   5529]
59 [556973 119753   5450]
60 [558572 118230   5374]
61 [560295 116569   5312]
62 [562091 114835   5250]
63 [563937 113032   5207]
64 [566001 111039   5136]
65 [568017 109078   5081]
66 [570301 106855   5020]
67 [572958 104280   4938]
68 [575503 101809   4864]
69 [578082  99301   4793]
70 [581318  96127   4731]
71 [584284  93223   4669]
72 [587787  89771   4618]
73 [591450  86160   4566]
74 [595413  82252   4511]
75 [599744  77991   4441]
76 [604413  

In [12]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/201'
#print('count_rand: mean ',np.mean(count_rand,axis=0), ' std ',np.std(count_rand,axis=0) )
#print('count_uni', np.mean(count_uni,axis=0), ' std ',np.std(count_uni,axis=0) )
#print('count_all',count_all)

count_rand,dist_rand = test_random(data_path,20)
count_uni, dist_uni = test_uniform(data_path,20)
count_all = test_all(data_path)

print('count_rand',np.mean(count_rand,axis=0),np.mean(dist_rand))
print('count_uni', np.mean(count_uni,axis=0),np.mean(dist_uni))
print('count_all',count_all)

[656993  11610  13573] 0.5468208944227511
[656668  11735  13773] 0.5388804182095404
[656794  11376  14006] 0.5299157503926889
[657542  11389  13245] 0.5603624009060023
[657511  11404  13261] 0.559686298167559
[656763  11510  13903] 0.5338416169172121
[657248  11359  13569] 0.5469820915321689
[657216  11415  13545] 0.5479512735326689
[655815  11559  14802] 0.5014187271990271
[657254  11646  13276] 0.559053931907201
[656042  11578  14556] 0.5098928276999175
[657788  11600  12788] 0.5803878636221458
[656056  11524  14596] 0.5084954782132091
[658110  11272  12794] 0.5801156792246366
[655135  11636  15405] 0.4817916260954236
[654596  11820  15760] 0.4709390862944162
[655051  11592  15533] 0.4778214124766626
[656442  11533  14201] 0.5226392507569889
[655906  11779  14491] 0.512179973776827
[656022  11459  14695] 0.5050697516161959
[656803  11523  13850] 0.535884476534296
[656696  11470  14010] 0.5297644539614561
[655783  11963  14430] 0.5143451143451143
[656090  11393  14693] 0.5051385013271

15 [658846  11194  12136] 0.6115688859591298
16 [659056  11191  11929] 0.6221812389974013
17 [659110  11185  11881] 0.6246948910024409
18 [659030  11199  11947] 0.6212438269021512
19 [659084  11201  11891] 0.6241695399882264
20 [659088  11204  11884] 0.6245371928643554
21 [659067  11204  11905] 0.6234355312893742
22 [658954  11198  12024] 0.6172654690618763
23 [658798  11201  12177] 0.609509731460951
24 [658846  11194  12136] 0.6115688859591298
25 [659056  11191  11929] 0.6221812389974013
26 [659110  11185  11881] 0.6246948910024409
27 [659030  11199  11947] 0.6212438269021512
28 [659084  11201  11891] 0.6241695399882264
29 [659088  11204  11884] 0.6245371928643554
30 [659067  11204  11905] 0.6234355312893742
31 [658954  11198  12024] 0.6172654690618763
32 [658798  11201  12177] 0.609509731460951
33 [658846  11194  12136] 0.6115688859591298
34 [659056  11191  11929] 0.6221812389974013
35 [659110  11185  11881] 0.6246948910024409
36 [659030  11199  11947] 0.6212438269021512
37 [659084  

29 [492989 153309  35878]
30 [494839 152341  34996]
31 [496632 151349  34195]
32 [498498 150348  33330]
33 [500351 149384  32441]
34 [502230 148399  31547]
35 [504112 147400  30664]
36 [505915 146419  29842]
37 [507697 145446  29033]
38 [509438 144447  28291]
39 [511061 143442  27673]
40 [512654 142457  27065]
41 [514243 141459  26474]
42 [515759 140450  25967]
43 [517283 139432  25461]
44 [518810 138356  25010]
45 [520272 137340  24564]
46 [521768 136250  24158]
47 [523355 135133  23688]
48 [524867 134049  23260]
49 [526480 132872  22824]
50 [527970 131745  22461]
51 [529601 130511  22064]
52 [531226 129317  21633]
53 [532857 128075  21244]
54 [534562 126790  20824]
55 [536248 125514  20414]
56 [538075 124144  19957]
57 [539953 122782  19441]
58 [541970 121286  18920]
59 [543969 119753  18454]
60 [545947 118230  17999]
61 [548029 116569  17578]
62 [550163 114835  17178]
63 [552274 113032  16870]
64 [554580 111039  16557]
65 [556869 109078  16229]
66 [559423 106855  15898]
67 [562237 1